### Drop unnecessary columns

In [ ]:
# import libraries
import pandas as pd
import seaborn as sns
import numpy as np
import pickle
import datetime
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
from sklearn import preprocessing 

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Feature Engineering

In [59]:
df_clean=train_df.drop(['visitId','geoNetwork.region','geoNetwork.metro','geoNetwork.city','domain',
                        'totals.transactions','totals.transactionRevenue','trafficSource.adwordsClickInfo.isVideoAd',
                        'trafficSource.source','trafficSource.campaign','trafficSource.referralPath','visitNumber',
                        'visitStartTime','trafficSource.keyword','trafficSource.adwordsClickInfo.gclId','_day','_year',
                       'trafficSource.adContent'],axis=1)

In [60]:
df_clean.head()

,channelGrouping,date,fullVisitorId,device.browser,device.operatingSystem,device.isMobile,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.networkDomain,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,trafficSource.medium,trafficSource.isTrueDirect,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.adNetworkType,_weekday,_month,_visitHour
0,Organic Search,2017-10-16,3162355547410993243,Firefox,Windows,False,desktop,Europe,Western Europe,Germany,(not set),1,1.0,1.0,1.0,1.0,NaN,NaN,organic,False,0,None,Unknown,0,10,19
1,Referral,2017-10-16,8934116514970143966,Chrome,Chrome OS,False,desktop,Americas,Northern America,United States,(not set),2,2.0,0.0,0.0,2.0,28.0,NaN,referral,False,0,None,Unknown,0,10,12
2,Direct,2017-10-16,7992466427990357681,Chrome,Android,True,mobile,Americas,Northern America,United States,windjammercable.net,2,2.0,0.0,1.0,1.0,38.0,NaN,(none),True,0,None,Unknown,0,10,19
3,Organic Search,2017-10-16,9075655783635761930,Chrome,Windows,False,desktop,Asia,Western Asia,Turkey,unknown.unknown,2,2.0,0.0,1.0,1.0,1.0,NaN,organic,False,0,None,Unknown,0,10,11
4,Organic Search,2017-10-16,6960673291025684308,Chrome,Windows,False,desktop,Americas,Central America,Mexico,prod-infinitum.com.mx,2,2.0,0.0,1.0,1.0,52.0,NaN,organic,False,0,None,Unknown,0,10,16


### Fill NaN values

In [61]:
df_clean['totals.timeOnSite'] = train_df['totals.timeOnSite'].fillna(0)
df_clean['totals.totalTransactionRevenue'] = train_df['totals.totalTransactionRevenue'].fillna(0.0)
df_clean['totals.pageviews']=df_clean['totals.pageviews'].fillna(0)
df_clean['totals.sessionQualityDim']=df_clean['totals.sessionQualityDim'].fillna(0)
df_clean['totals.sessionQualityDim']=df_clean['totals.sessionQualityDim'].astype('category')
df_clean['_weekday']=df_clean['_weekday'].astype('category')
df_clean['_month']=df_clean['_month'].astype('category')
df_clean['_visitHour']=df_clean['_visitHour'].astype('category')

In [62]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 928860 entries, 0 to 928859
Data columns (total 26 columns):
channelGrouping                                 928860 non-null category
date                                            928860 non-null datetime64[ns]
fullVisitorId                                   928860 non-null object
device.browser                                  928860 non-null category
device.operatingSystem                          928860 non-null category
device.isMobile                                 928860 non-null category
device.deviceCategory                           928860 non-null category
geoNetwork.continent                            928860 non-null category
geoNetwork.subContinent                         928860 non-null category
geoNetwork.country                              928860 non-null category
geoNetwork.networkDomain                        928860 non-null category
totals.hits                                     928860 non-null int32
totals.pagev

In [63]:
df_clean['month_4']=0
df_clean.loc[df_clean['_month']==4,'month_4']=1
df_clean['isTuesdays']=0
df_clean.loc[df_clean['_weekday']==2,'isTuesdays']=1
df_clean['BS_Firefox']=0
df_clean.loc[df_clean['device.browser']=='Firefox','BS_Firefox']=1
df_clean['BS_Chrome']=0
df_clean.loc[df_clean['device.browser']=='Chrome','BS_Chrome']=1
df_clean['BS_Safari']=0
df_clean.loc[df_clean['device.browser']=='Safari','BS_Safari']=1
df_clean['BS_IE']=0
df_clean.loc[df_clean['device.browser']=='Internet Explorer','BS_IE']=1
df_clean['BS_Android']=0
df_clean.loc[df_clean['device.browser']=='Android Webview','BS_Android']=1


df_clean['OS_Windows']=0
df_clean.loc[df_clean['device.operatingSystem']=='Windows','OS_Windows']=1
df_clean['OS_Macintosh']=0
df_clean.loc[df_clean['device.operatingSystem']=='Macintosh','OS_Macintosh']=1
df_clean['OS_Android']=0
df_clean.loc[df_clean['device.operatingSystem']=='Android','OS_Android']=1
df_clean['OS_iOS']=0
df_clean.loc[df_clean['device.operatingSystem']=='iOS','OS_iOS']=1


#subContinent1: Northern America (highest non-zero revenue)
df_clean['subCont_NorthernAmerica']=0
df_clean.loc[df_clean['geoNetwork.subContinent']=='Northern America','subCont_NorthernAmerica']=1

#subContinent2: Western Africa (highest mean revenue)
df_clean['subCont_Western Africa']=0
df_clean.loc[df_clean['geoNetwork.subContinent']=='Western Africa','subCont_Western Africa']=1
#country1: US (highest non-zero revenue)
df_clean['country_USA']=0
df_clean.loc[df_clean['geoNetwork.country']=='United States','country_USA']=1
#country2: Australia (highest mean revenue)
df_clean['country_Australia']=0
df_clean.loc[df_clean['geoNetwork.country']=='Australia','country_Australia']=1

#traffic medium
df_clean['medium_cpm']=0
df_clean.loc[df_clean['trafficSource.medium']=='cpm','medium_cpm']=1

In [64]:
#get dummies
df_clean=df_clean.join(pd.get_dummies(df_clean['channelGrouping'])).drop(['(Other)'],axis = 1)
df_clean=df_clean.join(pd.get_dummies(df_clean['_visitHour'],prefix='Hour')).drop(['Hour_1','Hour_4','Hour_5','Hour_6','Hour_7','Hour_16','Hour_17','Hour_18','Hour_19','Hour_20','Hour_21','Hour_22','Hour_23'],axis = 1)
df_clean=df_clean.join(pd.get_dummies(df_clean['device.isMobile'],prefix='Mobile',drop_first=True))
df_clean=df_clean.join(pd.get_dummies(df_clean['device.deviceCategory'])).drop(['tablet','mobile'],axis = 1)
df_clean=df_clean.join(pd.get_dummies(df_clean['geoNetwork.continent'])).drop(['(not set)'],axis = 1)
df_clean=df_clean.join(pd.get_dummies(df_clean['trafficSource.isTrueDirect'],prefix='isTrueDirect',drop_first=True))

In [65]:
df_clean['sessionQuality.100'] = pd.get_dummies(df_clean['totals.sessionQualityDim'])[100]
df_clean['Slot.RHS'] = pd.get_dummies(df_clean['trafficSource.adwordsClickInfo.slot'])['RHS']
df_clean['Slot.Top'] = pd.get_dummies(df_clean['trafficSource.adwordsClickInfo.slot'])['Top']
df_clean['Network.Content'] = pd.get_dummies(df_clean['trafficSource.adwordsClickInfo.adNetworkType'])['Content']
df_clean['Network.GSearch'] = pd.get_dummies(df_clean['trafficSource.adwordsClickInfo.adNetworkType'])['Google Search']
df_clean['Network.PSearch'] = pd.get_dummies(df_clean['trafficSource.adwordsClickInfo.adNetworkType'])['Search partners']

In [66]:
df_clean=df_clean.drop(['_month','channelGrouping','_weekday','device.operatingSystem','geoNetwork.country',
                       'geoNetwork.subContinent','geoNetwork.continent','geoNetwork.networkDomain','device.isMobile',
                       'device.deviceCategory','device.browser','trafficSource.medium','trafficSource.isTrueDirect',
                       '_visitHour','trafficSource.adwordsClickInfo.adNetworkType',
              'trafficSource.adwordsClickInfo.slot', 
              'totals.sessionQualityDim'],axis=1)

In [67]:
df_clean.head()

,date,fullVisitorId,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.timeOnSite,totals.totalTransactionRevenue,trafficSource.adwordsClickInfo.page,month_4,isTuesdays,BS_Firefox,BS_Chrome,BS_Safari,BS_IE,BS_Android,OS_Windows,OS_Macintosh,OS_Android,OS_iOS,subCont_NorthernAmerica,subCont_Western Africa,country_USA,country_Australia,medium_cpm,Affiliates,Direct,Display,Organic Search,Paid Search,Referral,Social,Hour_0,Hour_2,Hour_3,Hour_8,Hour_9,Hour_10,Hour_11,Hour_12,Hour_13,Hour_14,Hour_15,Mobile_True,desktop,Africa,Americas,Asia,Europe,Oceania,isTrueDirect_False,sessionQuality.100,Slot.RHS,Slot.Top,Network.Content,Network.GSearch,Network.PSearch
0,2017-10-16,3162355547410993243,1,1.0,1.0,1.0,0.0,0.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0
1,2017-10-16,8934116514970143966,2,2.0,0.0,0.0,28.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0
2,2017-10-16,7992466427990357681,2,2.0,0.0,1.0,38.0,0.0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
3,2017-10-16,9075655783635761930,2,2.0,0.0,1.0,1.0,0.0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0
4,2017-10-16,6960673291025684308,2,2.0,0.0,1.0,52.0,0.0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0


In [68]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 928860 entries, 0 to 928859
Data columns (total 57 columns):
date                                   928860 non-null datetime64[ns]
fullVisitorId                          928860 non-null object
totals.hits                            928860 non-null int32
totals.pageviews                       928860 non-null float64
totals.bounces                         928860 non-null float64
totals.newVisits                       928860 non-null float64
totals.timeOnSite                      928860 non-null float64
totals.totalTransactionRevenue         928860 non-null float64
trafficSource.adwordsClickInfo.page    928860 non-null int32
month_4                                928860 non-null int64
isTuesdays                             928860 non-null int64
BS_Firefox                             928860 non-null int64
BS_Chrome                              928860 non-null int64
BS_Safari                              928860 non-null int64
BS_IE          

### Produce Transaction Level of Dataset for Modeling

In [69]:
df_clean = df_clean.rename(columns={'totals.totalTransactionRevenue': 'revenue', 'fullVisitorId': 'id',
                                    'trafficSource.adwordsClickInfo.page': 'TS_adwordsClickInfo.page', 
                                    'Affiliates': 'CG_Affiliates', 'Direct': 'CG_Direct','Display':'CG_Display', 
                                    'Organic Search': 'CG_organicSearch',
                                   'Paid Search': 'CG_paidSearch' ,'Referral': 'CG_Referral',
                                   'Social': 'CG_Social', 'Mobile_True': 'device_Mobile',
                                   'desktop': 'device_Desktop', 'Africa': 'cont_Africa', 
                                   'Americas': 'cont_Americas', 'Asia': 'cont_Asia',
                                   'Europe': 'cont_Europe', 'Oceania': 'cont_Oceania',
                                   'isTrueDirect_False': 'TS_isTrueDirect', 'sessionQuality.100': 'TS_sessionQuality.100',
                                   'Slot.RHS': 'TS_Slot.RHS', 'Slot.Top': 'TS_Slot.Top', 
                                    'Network.Content': 'TS_Network.Content', 'Network.GSearch': 'TS_Network.GSearch',
                                    'Network.PSearch': 'TS_Network.PSearch'})

In [70]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 928860 entries, 0 to 928859
Data columns (total 57 columns):
date                        928860 non-null datetime64[ns]
id                          928860 non-null object
totals.hits                 928860 non-null int32
totals.pageviews            928860 non-null float64
totals.bounces              928860 non-null float64
totals.newVisits            928860 non-null float64
totals.timeOnSite           928860 non-null float64
revenue                     928860 non-null float64
TS_adwordsClickInfo.page    928860 non-null int32
month_4                     928860 non-null int64
isTuesdays                  928860 non-null int64
BS_Firefox                  928860 non-null int64
BS_Chrome                   928860 non-null int64
BS_Safari                   928860 non-null int64
BS_IE                       928860 non-null int64
BS_Android                  928860 non-null int64
OS_Windows                  928860 non-null int64
OS_Macintosh       

In [71]:
df_clean.to_pickle("trasaction_dataset.pkl")

In [ ]:
### Produce Aggregation to User level of Dataset for Modeling

#customer dataframe cf
cf1 = pd.DataFrame()  #float data
cf2 = pd.DataFrame()  #dummy data
cf3 = pd.DataFrame()  #target revenue

cf1 = df_clean.iloc[:,0:6]
cf1 = cf1.groupby(['id']).agg(['sum','mean'])
cf1.columns = ['_'.join(col).strip() for col in cf1.columns.values]
#cf1.head()

select_columns = [1]
select_columns.extend(range(8,57))
cf2 = df_clean[df_clean.columns[select_columns]]
cf2 = cf2.groupby(['id']).agg(['max'])
cf2.columns = ['_'.join(col).strip() for col in cf2.columns.values]
cf2.head()

cf3 = df_clean.groupby(['id'])['revenue'].agg(['sum'])
cf3['logRevenue'] = np.log(1+cf3['sum'])
cf3 = cf3.drop('sum', axis = 1)

# comebine customer dataset
cf = pd.concat([cf3, cf1, cf2], axis=1)
cf = cf.reset_index()
cf = pd.DataFrame(cf)
cf.shape

cf.head()

# save to pickle
cf.to_pickle("customer_dataset.pkl")

cf = pd.read_pickle('customer_dataset.pkl')

# drop time columns
time_columns = ['month_4_max','isTuesdays_max','Hour_0_max','Hour_2_max','Hour_3_max','Hour_8_max','Hour_9_max','Hour_10_max','Hour_11_max','Hour_12_max','Hour_13_max','Hour_14_max','Hour_15_max']

cf = cf.drop(time_columns, axis=1)

cf.info()